This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.1_tutorials/wallaroo-run-anywhere/edge-observability-low-no-connection).

## Edge Observability with No/Low Connection Tutorial

Wallaroo pipeline publishes are containerized versions of the pipeline, models and model steps, and the inference engine published to an Open Container Initiative (OCI) Registry.  Once published, they are used to deploy models to edge devices and used for inference requests.

Edge locations added to a pipeline published allow deployments on edge devices to connect with the Wallaroo Ops instance and transmit their inference results as part of the pipeline logs.

### Edge Location Observability

Inference request logs are stored locally in the edge device.  This storage space is configurarable during model deployment on the edge devices via the `LOCAL_INFERENCE_STORAGE` flag.  Inference logs are stored on the edge devices until the assigned storage capacity is reached.  At that point, logs are pruned on a "FIFO" - First In First Out method, where the first inference logs stored are the one pruned first.

In conditions where the edge device is unable to connect with the Wallaroo Ops instance for an extended period of time, logs are stored until connection is restored to the Wallaroo Ops instance.  When the connection is restored, logs are transmitted back to the Wallaroo Ops instance and added to the pipeline where the edge location is assigned to.

Edge inference logs have the inference DateTime is set to the DateTime the inference occured on the edge device, **not** when the logs are uploaded to the Wallaroo Ops.  Assays and other features that use inference logs are updated when they perform analyses with any new inference records uploaded by edge devices.  This insures that inference log information is synced across different edge devices without concern for when the data was uploaded, just when the inference occured.

## Goals

This tutorial will perform the following:

1. Upload and deploy a sample real estate house price estimation to a Wallaroo Ops instance.
1. Publish the pipeline and deploy to an edge device.
1. Perform sample inferences on the edge devices.
    1. Show inference logs from the local edge device.
    1. Show inference logs submitted to the Wallaroo Ops instance from the edge device.
1. Simulate a loss of communication on the Wallaroo Ops instance from the edge device. Perform additional sample inferences on the edge device.
    1. Show new inference logs on the edge device.
    1. Show new inference logs are not yet transmitted to the Wallaroo Ops instance.
1. Simulate a restoration of communition on the Wallaroo Ops instance and show the inference logs from the edge device are restored to the Wallaroo Ops instance.

### Resources

This tutorial provides the following:

* Models:
  * `./models/rf_model.onnx`: The champion model that predicts house prices.
* Data:
  * `./data/normal-inputs.df.json`: A set of inference requests across a span of different house types and values.
  * `./data/smallinputs.df.json`:  A set of inference requests that tend to generate house price predictions with low values.
  * `./data/biginputs.df.json`: A set of inference requests that tend to generate house price predictions with high values.

### Prerequisites

* A deployed Wallaroo instance with [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) and [Edge Observability enabled](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#set-edge-observability-service).
* The following Python libraries installed:
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
* A X64 Docker deployment to deploy the model on an edge location.


## Edge Deployment Preparation Steps

The following is used to set up the environment that is used for the pipeline publish replacement and in-line model updates on an edge tutorial.

In these steps, we will:

* Deploying a sample ML model used to determine house prices based on a set of input parameters.
* Publish the model deployment configuration to an OCI registry.
* Use the publish and set edge locations.
* Deploy the model in an edge location and perform sample inferences.

### Import Libraries

The first step will be to import our libraries, and set variables used through this tutorial.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import json
import time

workspace_name = 'edge-low-connection-demo'

# ignoring warnings for demonstration
import warnings
warnings.filterwarnings('ignore')

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

The following helper function is used to create a new workspace, or retrieve an existing one.

In [2]:
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = client.create_workspace(name)
    return workspace

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [4]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace.

Workspace names are unique across the Wallaroo instance.  If a user requests a workspace they have not been granted access to, an error message is returned.

In [5]:
workspace = get_workspace(workspace_name, wl)

wl.set_current_workspace(workspace)

{'name': 'edge-low-connection-demo', 'id': 6, 'archived': False, 'created_by': 'e790f98c-b2a7-403c-9616-c94f31a9f234', 'created_at': '2024-03-28T19:47:13.103565+00:00', 'models': [{'name': 'rf-house-price-estimator', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 3, 28, 19, 47, 13, 777445, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 3, 28, 19, 47, 13, 777445, tzinfo=tzutc())}], 'pipelines': [{'name': 'edge-low-connection-demo', 'create_time': datetime.datetime(2024, 3, 28, 19, 47, 13, 865125, tzinfo=tzutc()), 'definition': '[]'}]}

### Upload The Champion Model

For our example, we will upload the champion model that has been trained to derive house prices from a variety of inputs.  The model file is `rf_model.onnx`, and is uploaded with the name `rf-house-price-estimator`.

In [5]:
housing_model_control = (wl.upload_model("rf-house-price-estimator", 
                                        './models/rf_model.onnx', 
                                        framework=Framework.ONNX)
                                        .configure(tensor_fields=["tensor"])
                        )

### Build and Deploy the Pipeline

This pipeline is made to be an example of an existing situation where a model is deployed and being used for inferences in a production environment.  We'll call it `edge-inline-replacement-demonstration`, set `housing_model_control` as a pipeline step, then run a sample inference.

In [6]:
mainpipeline = wl.build_pipeline('edge-low-connection-demo')

# undeploy if already created earlier
mainpipeline.undeploy()
# clear the steps if used before
mainpipeline.clear()

mainpipeline.add_model_step(housing_model_control)

name,edge-low-connection-demo
created,2024-03-28 19:47:13.865125+00:00
last_updated,2024-03-28 19:47:13.865125+00:00
deployed,(none)
arch,None
accel,None
tags,
versions,2f23c073-50eb-4cbd-93c7-0330dadeaa9c
steps,
published,False


In [7]:
#minimum deployment config
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()

mainpipeline.deploy(deployment_config = deploy_config)

Waiting for deployment - this will take up to 45s ................ ok


name,edge-low-connection-demo
created,2024-03-28 19:47:13.865125+00:00
last_updated,2024-03-28 19:47:14.008128+00:00
deployed,True
arch,None
accel,None
tags,
versions,"b70c2549-0ab7-4937-a268-3a4e02825e72, 2f23c073-50eb-4cbd-93c7-0330dadeaa9c"
steps,rf-house-price-estimator
published,False


### Inference Request Testing

We'll use a single row inference request that should return a result of around `$700k`.

In [8]:
normal_input = pd.DataFrame.from_records({"tensor": [[4.0, 
                                                      2.5, 
                                                      2900.0, 
                                                      5505.0, 
                                                      2.0, 
                                                      0.0, 
                                                      0.0, 
                                                      3.0, 
                                                      8.0, 
                                                      2900.0, 
                                                      0.0, 
                                                      47.6063, 
                                                      -122.02, 
                                                      2970.0, 
                                                      5251.0, 
                                                      12.0, 
                                                      0.0, 
                                                      0.0]]})
result = mainpipeline.infer(normal_input)
display(result)

,time,in.tensor,out.variable,anomaly.count
0,2024-03-28 19:47:31.522,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.7],0


### View Pipeline Logs

We display the pipeline logs here to show the inference requests to the pipeline deployed in the Wallaroo Ops instance.

In [9]:
mainpipeline.logs()

,time,in.tensor,out.variable,anomaly.count
0,2024-03-28 19:47:31.522,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.7],0


### Edge Deployment

We can now deploy the original pipeline to an edge device.  This will require the following steps:

* Publish the pipeline:  Publishes the pipeline to the OCI registry.
* Add Edge:  Add the edge location to the pipeline publish.
* Deploy Edge:  Deploy the edge device with the edge location settings.

#### Publish Pipeline

Publishing the pipeline uses the pipeline `wallaroo.pipeline.publish()` command.  This requires that the Wallaroo Ops instance have [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) enabled.

The following publishes the pipeline to the OCI registry and displays the container details.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Edge Publication](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/).

In [10]:
pub = mainpipeline.publish()

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is publishing........ Published.


#### Add Edge Location

The edge location is added with the [`wallaroo.pipeline_publish.add_edge(name)`](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-reference-guide/pipeline_publish/#PipelinePublish.add_edge) method.  This returns the OCI registration information, and the `EDGE_BUNDLE` information.  The `EDGE_BUNDLE` data is a base64 encoded set of parameters for the pipeline that the edge device is associated with, the workspace, and other data.

For full details, see [Wallaroo SDK Essentials Guide: Pipeline Edge Publication: Edge Observability](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/#edge-observability).

For this example, we will add a location:

* `houseprice-edge-inline-demo`

In [11]:
edge_name_01 = "houseprice-low-connection-demonstration-01"
edge_publish_01 = pub.add_edge(edge_name_01)
display(edge_publish_01)

ID,1
Pipeline Name,edge-low-connection-demo
Pipeline Version,bf66ab5d-e4c0-4df3-aeef-a34510159c91
Status,Published
Engine URL,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4823
Pipeline URL,ghcr.io/wallaroolabs/doc-samples/pipelines/edge-low-connection-demo:bf66ab5d-e4c0-4df3-aeef-a34510159c91
Helm Chart URL,oci://ghcr.io/wallaroolabs/doc-samples/charts/edge-low-connection-demo
Helm Chart Reference,ghcr.io/wallaroolabs/doc-samples/charts@sha256:990e1361b20b4244359cb8014fd22e516a4114307a1ff06d0ee58605b4e05f48
Helm Chart Version,0.0.1-bf66ab5d-e4c0-4df3-aeef-a34510159c91
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 4.0, 'memory': '3Gi'}, 'requests': {'cpu': 4.0, 'memory': '3Gi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': None}, 'enginelb': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 0.2, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}}"
User Images,[]


#### DevOps Deployment

The following shows two methods of model deployments on edge devices.  For more details on model edge deployments with Wallaroo, see [Model Operations: Run Anywhere](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-run-anywhere/).

##### Docker Run Based Deployment

For the model deployment to the edge location via `docker run`, use the `Docker Run Command` displayed with the pipeline publish.    The following variables must be set first.

* `$REGISTRYURL`: The URL of the OCI registry service hosting the pipeline publish.
* `$OCI_USERNAME`: The username for the OCI registry service.
* `$OCI_PASSWORD`: The password or token for the OCI registry service.
* `$HELM_INSTALL_NAME`: The name of the Helm based installation for the edge deployment.
* `$HELM_INSTALL_NAMESPACE`: The name of the namespace to set the edge based deployment to.

Before deploying the pipeline, verify that Docker is able to connect to the OCI registry with the following command.

```bash
docker login -u $OCI_USERNAME --password $OCI_PASSWORD $REGISTRYURL
```

To add a specific port mapping, add the `-p {External Port}:8080` option to map the edge devices `{External Port}` to the Wallaroo edge deployment port `8080`.  For example, to map the external port `8080` to the internal port `8080`:

```bash
docker run -v $PERSISTENT_VOLUME_DIR:/persist \
    -p 8080:8080 \
    -e OCI_USERNAME=$OCI_USERNAME \
    -e OCI_PASSWORD=$OCI_PASSWORD \
    -e EDGE_BUNDLE=abcdefg\
    -e PIPELINE_URL=ghcr.io/wallaroolabs/doc-samples/pipelines/edge-inline-replacement-demonstration:96a926bf-bd64-4e88-ae27-9d5773fdc007 \
    -e CONFIG_CPUS=4 ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4806
```

##### Helm Based Deployment

For the model deployment to the edge location via `helm`, use the `Helm Install Command` displayed with the pipeline publish.  The following variables must be set first.

* `$REGISTRYURL`: The URL of the OCI registry service hosting the pipeline publish.
* `$OCI_USERNAME`: The username for the OCI registry service.
* `$OCI_PASSWORD`: The password or token for the OCI registry service.
* `$HELM_INSTALL_NAME`: The name of the Helm based installation for the edge deployment.
* `$HELM_INSTALL_NAMESPACE`: The name of the namespace to set the edge based deployment to.

Before deploying, verify `helm` has access to the target registry.  Using the variable above, this command is:

```bash
helm registry login $REGISTRYURL --username $OCI_USERNAME --password $OCI_PASSWORD
```

If the command succeeds, proceed.  If there are any issues, verify the settings above before continuing.

Once the helm registry service is set, deploy using the `Helm Install Command` from `edge_publish_01`.

Once deployed, use the following to port forward to the `helm` based deployment.

```bash
kubectl port-forward svc/engine-svc -n $HELM_INSTALL_NAMESPACE 8080 --address 0.0.0.0
```

#### Edge Inference Example

We will perform sample inferences on the **current** model edge deployment to demonstrate how the inferences change as the models are updated through the in-line edge deployments.

With the model deployed on the edge location, we will perform sample inferences on the edge deployment.  For this example, the hostname is `testboy.local`.  Adjust the hostname to fit your edge location deployment.

The following endpoints are available.

| Endpoint | Type | Description |
|---|---|---|
| `/models` | GET | Returns the models and model versions used in the edge deployment. |
| `/pipelines` | GET | Returns the pipeline ID and status of the pipeline used in the edge deployment. |
| `/infer` | POST | The inference request endpoint.  This accepts either an Apache Arrow table, or a JSON in pandas record format.

For more details on model edge deployments with Wallaroo, see [Model Operations: Run Anywhere](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-run-anywhere/).

First we'll retrieve the model.  We should see the same name we set when we uploaded it to the Wallaroo instance.

In [6]:
!curl testboy.local:8080/models

{"models":[{"model_version":{"name":"rf-house-price-estimator","visibility":"private","workspace_id":6,"conversion":{"python_version":"3.8","requirements":[],"framework":"onnx"},"id":1,"image_path":null,"status":"ready","task_id":null,"file_info":{"version":"11dc6474-302f-4711-8acb-01e0f21a0e0f","sha":"e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6","file_name":"rf_model.onnx"},"created_on_version":"2024.1.0"},"config":{"id":2,"model_version_id":1,"runtime":"onnx","filter_threshold":null,"tensor_fields":["tensor"],"input_schema":null,"output_schema":null,"batch_config":null,"sidekick_uri":null},"status":"Running"}]}

Now we'll retrieve the pipeline details, and verify the name is the same as set earlier and it is running.

In [7]:
!curl testboy.local:8080/pipelines

{"pipelines":[{"id":"edge-low-connection-demo","status":"Running"}]}

Now do the infer with the same DataFrame we used for our sample inference when the model was deployed in the Wallaroo instance.

In [56]:
import json

df = pd.DataFrame.from_records({"tensor": [[4.0, 
                                            2.5, 
                                            2900.0, 
                                            5505.0, 
                                            2.0, 
                                            0.0, 
                                            0.0, 
                                            3.0, 
                                            8.0, 
                                            2900.0, 
                                            0.0, 
                                            47.6063, 
                                            -122.02, 
                                            2970.0, 
                                            5251.0, 
                                            12.0, 
                                            0.0, 
                                            0.0]]})

data = df.to_dict(orient="records")

!curl -X POST testboy.local:8080/infer \
    -H "Content-Type: Content-Type: application/json; format=pandas-records" \
    --data '{json.dumps(data)}' > ./inferenceoutput.df.json

# get the dataframe from what we just did

df_result = pd.read_json('./inferenceoutput.df.json', orient="records")
df_result.loc[:, ['time', 'out', 'metadata']]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   628  100   500  100   128  23630   6049 --:--:-- --:--:-- --:--:-- 29904


,time,out,metadata
0,1711659492031,{'variable': [718013.7]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [60102, 362273], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"


In [59]:
pipeline.logs(dataset=['time', 'out', 'metadata'])

Pipeline log schema has changed over the logs requested 10 newest records retrieved successfully, newest record seen was at <datetime>. Please request additional records separately


,time,out.variable,metadata.last_model,metadata.pipeline_version,metadata.elapsed,metadata.dropped,metadata.partition
0,2024-03-28 20:51:20.650,[2016006.1],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[173059, 415640]",[],houseprice-low-connection-demonstration-01
1,2024-03-28 20:51:20.650,[1910824.0],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[173059, 415640]",[],houseprice-low-connection-demonstration-01
2,2024-03-28 20:51:20.650,[2005883.1],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[173059, 415640]",[],houseprice-low-connection-demonstration-01
3,2024-03-28 20:51:20.650,[1946437.8],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[173059, 415640]",[],houseprice-low-connection-demonstration-01
4,2024-03-28 20:51:20.650,[1689843.1],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[173059, 415640]",[],houseprice-low-connection-demonstration-01
5,2024-03-28 20:51:20.650,[1886959.2],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[173059, 415640]",[],houseprice-low-connection-demonstration-01
6,2024-03-28 20:51:20.650,[2002393.6],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[173059, 415640]",[],houseprice-low-connection-demonstration-01
7,2024-03-28 20:51:20.650,[1967344.1],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[173059, 415640]",[],houseprice-low-connection-demonstration-01
8,2024-03-28 20:51:20.650,[1514079.4],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[173059, 415640]",[],houseprice-low-connection-demonstration-01
9,2024-03-28 20:51:02.249,[712309.9],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8283612, 2478515]",[],houseprice-low-connection-demonstration-01


Let's do a request with more records.

In [90]:
!curl -X POST testboy.local:8080/infer \
    -H "Content-Type: Content-Type: application/json; format=pandas-records" \
    --data @./data/smallinputs.df.json > ./inferenceoutput2.df.json

# get the dataframe from what we just did

df_result = pd.read_json('./inferenceoutput2.df.json', orient="records")
df_result.tail(10).loc[:, ['time', 'out', 'metadata']]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100  626k  100  505k  100  121k  1568k   376k --:--:-- --:--:-- --:--:-- 1947k


,time,out,metadata
990,1711661272029,{'variable': [437177.84]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [8142688, 2787720], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
991,1711661272029,{'variable': [236238.66]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [8142688, 2787720], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
992,1711661272029,{'variable': [276046.53]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [8142688, 2787720], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
993,1711661272029,{'variable': [758714.2]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [8142688, 2787720], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
994,1711661272029,{'variable': [627853.5]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [8142688, 2787720], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
995,1711661272029,{'variable': [450867.56]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [8142688, 2787720], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
996,1711661272029,{'variable': [553463.4]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [8142688, 2787720], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
997,1711661272029,{'variable': [241330.19]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [8142688, 2787720], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
998,1711661272029,{'variable': [1295531.3]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [8142688, 2787720], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
999,1711661272029,{'variable': [712309.9]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [8142688, 2787720], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"


Get the most recent logs to verify the edge records were transmitted.  We'll see in the `metadata.partition` section the name matching our edge location.

In [94]:
pipeline.logs(dataset=['time', 'out', 'metadata'])

,time,out.variable,metadata.last_model,metadata.pipeline_version,metadata.elapsed,metadata.dropped,metadata.partition
0,2024-03-28 21:27:52.029,[712309.9],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
1,2024-03-28 21:27:52.029,[1295531.2],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
2,2024-03-28 21:27:52.029,[241330.19],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
3,2024-03-28 21:27:52.029,[553463.4],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
4,2024-03-28 21:27:52.029,[450867.56],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
...,...,...,...,...,...,...,...
95,2024-03-28 21:27:52.029,[450867.56],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
96,2024-03-28 21:27:52.029,[715530.0],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
97,2024-03-28 21:27:52.029,[837085.4],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
98,2024-03-28 21:27:52.029,[630865.6],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01


### Simulate Connection Loss

To simulate connection loss, we'll use a firewall rule.  The edge device is on a Ubuntu Linux OS, so we can use the following to block access to the Wallaroo Ops instance while leaving it open to others use.

```bash
sudo ufw deny out to $OPS_INSTANCE
```

We'll perform another set of inferences, this time from the `./data/biginputs.df.json` list.  When we retrieve the pipeline logs again from the Wallaroo Ops instance, we should not see any additional records.

In [96]:
!curl -X POST testboy.local:8080/infer \
    -H "Content-Type: Content-Type: application/json; format=pandas-records" \
    --data @./data/biginputs.df.json > ./inferenceoutput2.df.json

# get the dataframe from what we just did

df_result = pd.read_json('./inferenceoutput2.df.json', orient="records")
df_result.tail(10).loc[:, ['time', 'out', 'metadata']]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5822  100  4675  100  1147   188k  47430 --:--:-- --:--:-- --:--:--  236k


,time,out,metadata
0,1711661353215,{'variable': [1514079.4]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [108254, 451857], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
1,1711661353215,{'variable': [1967344.1]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [108254, 451857], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
2,1711661353215,{'variable': [2002393.6]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [108254, 451857], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
3,1711661353215,{'variable': [1886959.3]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [108254, 451857], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
4,1711661353215,{'variable': [1689843.1]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [108254, 451857], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
5,1711661353215,{'variable': [1946437.8]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [108254, 451857], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
6,1711661353215,{'variable': [2005883.1]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [108254, 451857], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
7,1711661353215,{'variable': [1910824.0]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [108254, 451857], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"
8,1711661353215,{'variable': [2016006.1]},"{'last_model': '{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}', 'pipeline_version': 'bf66ab5d-e4c0-4df3-aeef-a34510159c91', 'elapsed': [108254, 451857], 'dropped': [], 'partition': 'houseprice-low-connection-demonstration-01'}"


In [97]:
pipeline.logs(dataset=['time', 'out', 'metadata'])

,time,out.variable,metadata.last_model,metadata.pipeline_version,metadata.elapsed,metadata.dropped,metadata.partition
0,2024-03-28 21:27:52.029,[712309.9],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
1,2024-03-28 21:27:52.029,[1295531.2],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
2,2024-03-28 21:27:52.029,[241330.19],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
3,2024-03-28 21:27:52.029,[553463.4],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
4,2024-03-28 21:27:52.029,[450867.56],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
...,...,...,...,...,...,...,...
95,2024-03-28 21:27:52.029,[450867.56],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
96,2024-03-28 21:27:52.029,[715530.0],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
97,2024-03-28 21:27:52.029,[837085.4],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01
98,2024-03-28 21:27:52.029,[630865.6],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[8142688, 2787720]",[],houseprice-low-connection-demonstration-01


We'll now reenable communication, then wait 60 seconds and check the pipeline inference logs to see if communication is restored.

In [99]:
import time
time.sleep(60)

import json

df = pd.DataFrame.from_records({"tensor": [[4.0, 
                                            2.5, 
                                            2900.0, 
                                            5505.0, 
                                            2.0, 
                                            0.0, 
                                            0.0, 
                                            3.0, 
                                            8.0, 
                                            2900.0, 
                                            0.0, 
                                            47.6063, 
                                            -122.02, 
                                            2970.0, 
                                            5251.0, 
                                            12.0, 
                                            0.0, 
                                            0.0]]})

data = df.to_dict(orient="records")

!curl -X POST testboy.local:8080/infer \
    -H "Content-Type: Content-Type: application/json; format=pandas-records" \
    --data '{json.dumps(data)}' > ./inferenceoutput.df.json

# get the dataframe from what we just did

df_result = pd.read_json('./inferenceoutput.df.json', orient="records")
df_result.loc[:, ['time', 'out', 'metadata']]


pipeline.logs(dataset=['time', 'out', 'metadata'])

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   628  100   500  100   128   2171    555 --:--:-- --:--:-- --:--:--  2718


Pipeline log schema has changed over the logs requested 10 newest records retrieved successfully, newest record seen was at <datetime>. Please request additional records separately


,time,out.variable,metadata.last_model,metadata.pipeline_version,metadata.elapsed,metadata.dropped,metadata.partition
0,2024-03-28 21:33:29.999,[718013.7],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[58509, 356783]",[],houseprice-low-connection-demonstration-01
1,2024-03-28 21:29:13.215,[2016006.1],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[108254, 451857]",[],houseprice-low-connection-demonstration-01
2,2024-03-28 21:29:13.215,[1910824.0],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[108254, 451857]",[],houseprice-low-connection-demonstration-01
3,2024-03-28 21:29:13.215,[2005883.1],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[108254, 451857]",[],houseprice-low-connection-demonstration-01
4,2024-03-28 21:29:13.215,[1946437.8],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[108254, 451857]",[],houseprice-low-connection-demonstration-01
5,2024-03-28 21:29:13.215,[1689843.1],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[108254, 451857]",[],houseprice-low-connection-demonstration-01
6,2024-03-28 21:29:13.215,[1886959.2],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[108254, 451857]",[],houseprice-low-connection-demonstration-01
7,2024-03-28 21:29:13.215,[2002393.6],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[108254, 451857]",[],houseprice-low-connection-demonstration-01
8,2024-03-28 21:29:13.215,[1967344.1],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[108254, 451857]",[],houseprice-low-connection-demonstration-01
9,2024-03-28 21:29:13.215,[1514079.4],"{""model_name"":""rf-house-price-estimator"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}",bf66ab5d-e4c0-4df3-aeef-a34510159c91,"[108254, 451857]",[],houseprice-low-connection-demonstration-01
